# Text Classification

## 데이터 준비(복습)

In [1]:
from nltk.corpus import movie_reviews
fileids = movie_reviews.fileids() #movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] #file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 
#file id를 이용해 label로 사용할 category 즉 positive와 negative 정보를 순서대로 가져옴

print('Reviews count:', len(reviews))
print('Length of the first review:', len(reviews[0]))
print('Labels:', set(categories))

Reviews count: 2000
Length of the first review: 4043
Labels: {'pos', 'neg'}


### train set과 test set의 분리

In [2]:
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
X_train, X_test, y_train, y_test = train_test_split(reviews, categories, test_size=0.2, random_state=10)

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

Train set count:  1600
Test set count:  400


## TFIDF 변환

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
#sklearn에서 제공하는 TfidfVectorizer를 이용
tfidf = TfidfVectorizer().fit(X_train) # X_train을 이용하여 vectorizer를 학습
tfidf

TfidfVectorizer()

In [4]:
X_train_tfidf = tfidf.transform(X_train) #학습된 vectorizer를 통한 train set
X_train_tfidf.shape # 1600 (review 수) x 36310 (전체 corpus에서 사용된 단어의 수) 크기로 vector set이 생성됨
# matrix 안의 값은 해당 tfidf score임

(1600, 36310)

In [6]:
tfidf = TfidfVectorizer(max_features=2000).fit(X_train) #사용된 단어의 수가 너무 많은 경우, max_feature를 제한하여 학습이 가능
tfidf

TfidfVectorizer(max_features=2000)

In [7]:
X_train_tfidf = tfidf.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_tfidf.shape) # max값 적용됨
X_test_tfidf = tfidf.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_tfidf.shape) # max값 적용됨

Train set dimension: (1600, 2000)
Test set dimension: (400, 2000)


## Naive Bayse Classifier (Scikit)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000).fit(X_train) # max_feature 제한
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # max 값 적용
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

Train set dimension: (1600, 2000)
Test set dimension: (400, 2000)


In [10]:
from sklearn.naive_bayes import MultinomialNB 
NB_clf = MultinomialNB() # 분류기

NB_clf.fit(X_train_cv, y_train) #train set을 이용하여 분류기(classifier)를 학습

MultinomialNB()

In [11]:
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train))) #train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test))) #test set에 대한 예측정확도를 확인

Train set score: 0.864
Test set score: 0.775


In [12]:
#여러 문장에 대해 count vectorier로 변환 후 학습된 분류기로 결과를 예측
print(NB_clf.predict(cv.transform(['the story was unimaginative', 'the plot was ludicrous', 'kate winslet is accessible'])))

#위 첫째 문장에서 story를 actor로 바꿔서 예측
print(NB_clf.predict(cv.transform(['the actor was unimaginative'])))
#동일한 형용사라도 대상에 따라 결과가 바뀔 수 있음

['pos' 'neg' 'pos']
['neg']


## Logistic Regression (Scikit)

In [13]:
from sklearn.linear_model import LogisticRegression #sklearn이 제공하는 logistic regression을 사용

#count vector에 대해 regression을 해서 NB와 비교
LR_clf_cv = LogisticRegression() #분류기 선언
LR_clf_cv.fit(X_train_cv, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(LR_clf_cv.score(X_train_cv, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf_cv.score(X_test_cv, y_test))) # test data에 대한 예측정확도
# 더 많은 train data를 통해 tfidf의 값을 좋게 해줘야함

Train set score: 1.000
Test set score: 0.825


C:\Users\HyunSeop\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
LR_clf = LogisticRegression() #분류기 선언
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도

Train set score: 0.917
Test set score: 0.820


## Ridge regression

In [15]:
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier() #릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.974
Test set score: 0.838


## Lasso regression

In [16]:
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear') # Lasso는 동일한 LogisticRegression을 사용하면서 매개변수로 지정
lasso_clf.fit(X_train_tfidf, y_train) # train data로 학습
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1]) 
# 실제로 영향을 미치는 단어 개수 파악 가능

Train set score: 0.804
Test set score: 0.770
Used features count: 78 out of 2000


In [17]:
print(len(tfidf.vocabulary_)) # tfidf에 사용된 단어의 수
tfidf_voca = tfidf.get_feature_names() # tfidf의단어이름
tfidf_voca[:10] # 앞 10개를 출력

2000


['000', '10', '100', '13', '15', '1995', '1996', '1997', '1998', '1999']

In [18]:
print((lasso_clf.coef_ != 0)[0].tolist()[:100]) 
# coefficient의 사용여부 파악 100개까지

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]


In [19]:
selected_features = []
for i, sign in enumerate((lasso_clf.coef_ != 0)[0].tolist()):
    if sign: selected_features.append(tfidf_voca[i]) 
#사용여부가 True일 경우 단어를 selected_features에 저장

In [20]:
print(selected_features) 
len(selected_features)
# positive, negative에 결정적 영향을 미치는 단어들

['aliens', 'also', 'and', 'any', 'as', 'attempt', 'awful', 'bad', 'batman', 'boring', 'cameron', 'carpenter', 'could', 'definitely', 'director', 'dull', 'even', 'excellent', 'family', 'great', 'hanks', 'harry', 'have', 'he', 'her', 'here', 'his', 'in', 'is', 'jackie', 'julie', 'lame', 'least', 'lebowski', 'life', 'looks', 'many', 'matrix', 'mess', 'most', 'movie', 'mulan', 'no', 'nothing', 'off', 'on', 'only', 'perfect', 'perfectly', 'plot', 'political', 'poor', 'quite', 'reason', 'ridiculous', 'scream', 'script', 'seagal', 'see', 'seen', 'shrek', 'so', 'stupid', 'supposed', 'the', 'then', 'there', 'this', 'to', 'truman', 'unfortunately', 'very', 'war', 'waste', 'well', 'west', 'why', 'worst']


78

In [21]:
tfidf_voca[(lasso_clf.coef_ != 0)[0].tolist().index(True)]
# 선택된 단어들 중 첫번째 단어

'aliens'

In [22]:
print([tfidf_voca[i] for i, j in enumerate((lasso_clf.coef_ != 0)[0].tolist()) if j]) 
#선택된 단어들을 출력하는 또다른 방법

['aliens', 'also', 'and', 'any', 'as', 'attempt', 'awful', 'bad', 'batman', 'boring', 'cameron', 'carpenter', 'could', 'definitely', 'director', 'dull', 'even', 'excellent', 'family', 'great', 'hanks', 'harry', 'have', 'he', 'her', 'here', 'his', 'in', 'is', 'jackie', 'julie', 'lame', 'least', 'lebowski', 'life', 'looks', 'many', 'matrix', 'mess', 'most', 'movie', 'mulan', 'no', 'nothing', 'off', 'on', 'only', 'perfect', 'perfectly', 'plot', 'political', 'poor', 'quite', 'reason', 'ridiculous', 'scream', 'script', 'seagal', 'see', 'seen', 'shrek', 'so', 'stupid', 'supposed', 'the', 'then', 'there', 'this', 'to', 'truman', 'unfortunately', 'very', 'war', 'waste', 'well', 'west', 'why', 'worst']


## LSA

### Scikit-learn LSA

In [24]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)  # component 수 지정
svd.fit(X_train_tfidf)  # train data로 학습
print(svd.explained_variance_ratio_)  # 각 component의 분산비율
print(svd.explained_variance_ratio_.sum()) # component들의 분산의 합
# 34%정도
print(svd.singular_values_) 
print(svd.components_.shape)

[0.01525261 0.01669138 0.0133583  0.01042773 0.00741403 0.00730082
 0.00677966 0.00583257 0.00555167 0.00520324 0.00493018 0.00489498
 0.00458197 0.0044574  0.00432849 0.00421975 0.0041921  0.00413251
 0.00403416 0.00398954 0.00388074 0.00379435 0.00377392 0.00370049
 0.00361687 0.00358558 0.00350955 0.00340951 0.0033149  0.00327589
 0.00321078 0.00319169 0.00315556 0.00311055 0.00304756 0.00302892
 0.00300949 0.00297231 0.00293611 0.00292412 0.00290208 0.00287548
 0.0028342  0.00280742 0.00276097 0.00274277 0.00270645 0.00268513
 0.00267427 0.00265801 0.00258256 0.00257419 0.00256683 0.00255658
 0.00251155 0.00250159 0.00248046 0.00247207 0.00245957 0.00245289
 0.00241614 0.0023879  0.00236025 0.00234094 0.00233893 0.00231345
 0.00229516 0.0022807  0.00227705 0.00224271 0.00222958 0.00222203
 0.00220312 0.00219438 0.00218776 0.0021722  0.00215608 0.00215051
 0.00213564 0.00212784 0.00211271 0.00208695 0.00207613 0.00206077
 0.00204613 0.00203201 0.00200931 0.00199865 0.00199714 0.0019

In [25]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 2,000개의 feature로부터 feature extract (dimension reduce)
print(X_train_tfidf.shape) #축소하기 전
print(X_train_svd.shape) #축소된 후, 2000 -> 100개로 줄어 있음

(1600, 2000)
(1600, 100)


### LSA를 이용한 Logistic Regression

In [26]:
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train) #축소된 값으로 학습
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))

Train set score: 0.838
Test set score: 0.790
